# Artificial Neural Networks

### Importing the libraries

In [1]:
import numpy as np
# don't need matplotlib
import pandas as pd
import tensorflow as tf

2023-10-09 16:59:12.108866: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-09 16:59:12.137546: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-09 16:59:12.137592: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-09 16:59:12.137611: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-09 16:59:12.143360: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-09 16:59:12.144253: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
tf.__version__

'2.14.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [3]:
dataset = pd.read_csv("Churn_Modelling.csv")

# independent variables (excluding the first three columns: entry number, customer ID, customer name)
# customer name is truly irrelevant to the model?
X = dataset.iloc[:, 3:-1].values
# dependent variable (variable to be predicted; rightmost column)
y = dataset.iloc[:, -1].values

In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


### Encoding Categorical Data

#### Label encoding the "Gender" column

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
# encoding just the gender column

In [6]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


#### One Hot Encoding the "Geography" column

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(), [1])], remainder="passthrough")
X = np.array(ct.fit_transform(X))

In [8]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the data into the training set and test set

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

### Feature Scaling

In [10]:
# Feature scaling is absolutely compulsory for all data in deep learning, even binary variables
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train= sc.fit_transform(X_train) 
X_test = sc.transform(X_test)

## Part 2: Building the ANN

### Initializing the ANN

In [11]:
# Keras used to be its own library but was integrated into TensorFlow as of 2.0
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [12]:
# there is no perfect answer for the best number of neurons in the hidden layer. It requires experimentation.
ann.add(tf.keras.layers.Dense(units=6, activation="relu")) # relu = rectifier

### Adding the second hidden layer

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

### Adding the output layer

In [14]:
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid")) # sigmoid returns a probability of being either 0 or 1
# The outcome is binary, so one neuron is enough. For classification tasks you would need more


## Part 3 - Training the ANN

### Compiling the ANN

In [15]:
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
# adam = a sochastic gradient descent optimizer
# binary cross entropy is always used for binary dependent variables
# you can enter many metrics, but we are ony using one

### Training the ANN on the Training set

In [16]:
ann.fit(X_train, y_train, batch_size= 32, epochs=100) # 32 is the default batch size value

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.6001 - accuracy: 0.7221
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4747 - accuracy: 0.8027
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4286 - accuracy: 0.8236
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4011 - accuracy: 0.8347
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3841 - accuracy: 0.8406
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3722 - accuracy: 0.8469
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3630 - accuracy: 0.8518
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3566 - accuracy: 0.8535
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3527 - accuracy: 0.8535
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3490 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

Homework: Use the ANN model to predict if the customer with the following information will leave the bank:

Geography: France  
Credit score: 600  
Gender: Male  
Age: 40  
Tenure: 3 years  
Balance: 60000  
Number of products: 2  
Does this customer have a credit card? Yes  
Is this customer an active member: yes  
Estimated Salary: $50000  
So, should we say goodbye to the customer?

In [18]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5) # converts output above 0.5 to true
# Must have the right number of elements!

1/1 [==============================] - 0s 14ms/step
[[False]]


### Predicting the Test set results

In [21]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1 )), 1)) 

 1/63 [..............................] - ETA: 0s

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [22]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

[[1520   65]
 [ 207  208]]


0.864